In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import os
import datetime
pd.options.display.max_rows = 20

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

import warnings
warnings.filterwarnings(action='ignore')

## 아래는 matplotlib 한국어 표시해주는 코드

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

from IPython.display import Image
from IPython.core.display import HTML 

# 유동인구

In [ ]:
floating_pop = pd.read_csv('귀미한/핫플비핫플_유동인구.csv', encoding='cp949')

In [ ]:
floating_pop

## 유동인구 - 구별 유동인구 연령대와 젊은이비율을 cutoff로 끊어줌

In [4]:
# quantile([0.25,0.5,0.75])

floating_pop.quantile([0.75, 0.25])[['연령_20대','연령_30대','젊은이비율']]

,연령_20대,연령_30대,젊은이비율
0.75,51498875.0,56781710.0,40.85
0.25,34779020.0,35864545.0,36.00


In [5]:

high_a_2 = floating_pop.quantile([0.75, 0.25])[['연령_20대','연령_30대','젊은이비율']].iloc[0,0]
high_a_3 = floating_pop.quantile([0.75, 0.25])[['연령_20대','연령_30대','젊은이비율']].iloc[0,1]
high_r = floating_pop.quantile([0.75, 0.25])[['연령_20대','연령_30대','젊은이비율']].iloc[0,2]


low_a_2 = floating_pop.quantile([0.75, 0.25])[['연령_20대','연령_30대','젊은이비율']].iloc[1,0]
low_a_3 = floating_pop.quantile([0.75, 0.25])[['연령_20대','연령_30대','젊은이비율']].iloc[1,1]
low_r = floating_pop.quantile([0.75, 0.25])[['연령_20대','연령_30대','젊은이비율']].iloc[1,2]

In [6]:
# 상위 25, 하위 25%를 기준으로 삼음 1)20,30대 유동인구 수 

high_list = floating_pop.query(f'젊은이비율>= {high_r} and 연령_20대 >= {high_a_2} and 연령_30대 >= {high_a_3}').구.unique()
low_list = floating_pop.query(f'젊은이비율<= {low_r} and 연령_20대 <= {low_a_2} and 연령_30대 >= {low_a_3}').구.unique()

print("*"*50)
print(f'상위 25%의 20,30대 유동인구 수와 비중이 나타는 지역은 : {high_list}')
print(f'하위 25%의 20,30대 유동인구 수와 비중이 나타는 지역은 : {low_list}')

**************************************************
상위 25%의 20,30대 유동인구 수와 비중이 나타는 지역은 : ['강남구' '관악구' '서초구' '마포구']
하위 25%의 20,30대 유동인구 수와 비중이 나타는 지역은 : ['중랑구']


In [7]:
# 상위 25, 하위 25%를 기준으로 삼음 1)20대 유동인구 수

high_list = floating_pop.query(f'젊은이비율>= {high_r} and 연령_20대 >= {high_a_2}').구.unique()
low_list = floating_pop.query(f'젊은이비율<= {low_r} and 연령_20대 <= {low_a_2}').구.unique()

print("*"*50)
print(f'상위 25%의 20대 유동인구 수와 비중이 나타는 지역은 : {high_list}')
print(f'하위 25%의 20대 유동인구 수와 비중이 나타는 지역은 : {low_list}')


**************************************************
상위 25%의 20대 유동인구 수와 비중이 나타는 지역은 : ['강남구' '관악구' '마포구' '서초구']
하위 25%의 20대 유동인구 수와 비중이 나타는 지역은 : ['강북구' '도봉구' '중랑구']


## 젊은이 비율, 지역별 월별 변화추이

In [8]:
df = floating_pop.loc[floating_pop['월']==2][['구','젊은이비율']]
df.columns = ['구', '2월_젊은이비율']

In [9]:
for i in range(3,9):
    df1 = floating_pop.loc[floating_pop['월']==i][['구','젊은이비율']]
    df1.columns = [f'{i}구', f'{i}월_젊은이비율']
    df = df.merge(df1, left_on='구',right_on=f'{i}구', how='left').drop(f'{i}구',axis=1)

In [10]:
df.to_csv('귀미한/핫플비핫플_유동인구_월별젊은이비율.csv', encoding='cp949')

# 추정매출과 상권변화지표

In [11]:
jipyo = pd.read_csv('귀미한/핫플비핫플_상권변화지표.csv', encoding='cp949')
exp_sales = pd.read_csv('귀미한/핫플비핫플_추정매출.csv', encoding = 'cp949')

In [12]:
jipyo.head()

,기준_년_코드,기준_분기_코드,행정동_코드_명,상권_변화_지표_명
0,2018,1,가락1동,정체
1,2018,1,가락2동,상권축소
2,2018,1,가락본동,다이나믹
3,2018,1,가리봉동,정체
4,2018,1,가산동,다이나믹


In [13]:
exp_sales.head()

,기준_년_코드,기준_분기_코드,시군구명,읍면동명,당월_매출_금액
0,2020,1,강남구,개포2동,12624371008
1,2020,1,강남구,개포4동,10875283987
2,2020,1,강남구,논현1동,48629212194
3,2020,1,강남구,논현2동,60419179286
4,2020,1,강남구,대치1동,11108924755


## 구,동 정보가 든 기준 테이블 jns 생성

In [29]:
jns = exp_sales.drop(['기준_년_코드', '기준_분기_코드', '당월_매출_금액'], axis=1)
jns

,시군구명,읍면동명
0,강남구,개포2동
1,강남구,개포4동
2,강남구,논현1동
3,강남구,논현2동
4,강남구,대치1동
...,...,...
743,중랑구,상봉제1동
744,중랑구,상봉제2동
745,중랑구,신내1동
746,중랑구,중화제1동


### 2019년에 추정된 1분기, 2분기 추정매출을 활용하여, 해당 상권의 매출을 파악 
(코로나 영향이 반영되지 않은 파일이기 때문에, 코로나 영향을 제외한 동별 매출의 동향을 파악하기 용이함)

In [31]:
for i in range(1,3):
    monthly = exp_sales.loc[exp_sales['기준_분기_코드'] == i].drop(['기준_년_코드', '기준_분기_코드'], axis=1)
    monthly.columns = [f'{i}구',f'{i}동', f'추정매출_{i}분기']
    jns = jns.merge(monthly, left_on = ['시군구명', '읍면동명'], right_on = [f'{i}구', f"{i}동"], how = "left").drop([f'{i}구', f'{i}동'],axis=1)
jns

,시군구명,읍면동명,추정매출_1분기,추정매출_2분기
0,강남구,개포2동,12624371008,13182044004
1,강남구,개포4동,10875283987,12128061682
2,강남구,논현1동,48629212194,48167311704
3,강남구,논현2동,60419179286,61512365973
4,강남구,대치1동,11108924755,11195808375
...,...,...,...,...
743,중랑구,상봉제1동,139860794,155099583
744,중랑구,상봉제2동,8923125271,10204694695
745,중랑구,신내1동,4372497690,4921250494
746,중랑구,중화제1동,1203728896,1257438224


### 데이터 변환 필요. ~제1동 vs ~1동으로

In [32]:
jns.to_csv('귀미한/동변화필요.csv', encoding='cp949')

In [33]:
for k in range(len(jns['읍면동명'])):
    if (jns['읍면동명'][k] != '제기동') & (jns['읍면동명'][k] != '홍제1동')& (jns['읍면동명'][k] != '홍제2동')& (jns['읍면동명'][k] != '홍제3동'):
        jns['읍면동명'][k] = jns['읍면동명'][k].replace("제","")


In [34]:
jns

,시군구명,읍면동명,추정매출_1분기,추정매출_2분기
0,강남구,개포2동,12624371008,13182044004
1,강남구,개포4동,10875283987,12128061682
2,강남구,논현1동,48629212194,48167311704
3,강남구,논현2동,60419179286,61512365973
4,강남구,대치1동,11108924755,11195808375
...,...,...,...,...
743,중랑구,상봉1동,139860794,155099583
744,중랑구,상봉2동,8923125271,10204694695
745,중랑구,신내1동,4372497690,4921250494
746,중랑구,중화1동,1203728896,1257438224


### 상권변화지표 2018-2020 정보 정리

In [35]:
jipyo.head()

,기준_년_코드,기준_분기_코드,행정동_코드_명,상권_변화_지표_명
0,2018,1,가락1동,정체
1,2018,1,가락2동,상권축소
2,2018,1,가락본동,다이나믹
3,2018,1,가리봉동,정체
4,2018,1,가산동,다이나믹


In [36]:
for i in range(1,3):
    for j in range(2018,2021):
        new = jipyo.loc[(jipyo['기준_년_코드'] == j) & (jipyo['기준_분기_코드'] == i)].drop(['기준_년_코드', '기준_분기_코드'], axis=1)
        new.columns = ['구동',f'{j}년{i}분기_상권변화지표']
        jns = jns.merge(new, left_on = '읍면동명', right_on = '구동', how = "left").drop(f'구동',axis=1)


In [38]:
jns.columns

Index(['시군구명', '읍면동명', '추정매출_1분기', '추정매출_2분기', '2018년1분기_상권변화지표',
       '2019년1분기_상권변화지표', '2020년1분기_상권변화지표', '2018년2분기_상권변화지표',
       '2019년2분기_상권변화지표', '2020년2분기_상권변화지표'],
      dtype='object')

In [39]:
jns = jns[['시군구명', '읍면동명', '추정매출_1분기', '추정매출_2분기', 
           '2018년1분기_상권변화지표','2018년2분기_상권변화지표',
       '2019년1분기_상권변화지표',  '2019년2분기_상권변화지표',
           '2020년1분기_상권변화지표', '2020년2분기_상권변화지표']]

In [40]:
jns

,시군구명,읍면동명,추정매출_1분기,추정매출_2분기,2018년1분기_상권변화지표,2018년2분기_상권변화지표,2019년1분기_상권변화지표,2019년2분기_상권변화지표,2020년1분기_상권변화지표,2020년2분기_상권변화지표
0,강남구,개포2동,12624371008,13182044004,정체,정체,정체,정체,정체,정체
1,강남구,개포4동,10875283987,12128061682,상권축소,상권축소,상권축소,상권축소,상권축소,상권축소
2,강남구,논현1동,48629212194,48167311704,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
3,강남구,논현2동,60419179286,61512365973,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
4,강남구,대치1동,11108924755,11195808375,상권확장,상권확장,다이나믹,다이나믹,다이나믹,다이나믹
...,...,...,...,...,...,...,...,...,...,...
743,중랑구,상봉1동,139860794,155099583,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
744,중랑구,상봉2동,8923125271,10204694695,상권축소,상권축소,다이나믹,다이나믹,다이나믹,다이나믹
745,중랑구,신내1동,4372497690,4921250494,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
746,중랑구,중화1동,1203728896,1257438224,상권축소,상권축소,다이나믹,다이나믹,상권축소,다이나믹


In [41]:
jns.to_csv('귀미한/핫플비핫플_지표매출모음.csv', encoding='cp949')

## 젊은 유동인구 높은 지역와 낮은 지역의 지표, 매출만 살펴보기

### 우선 1,2분기 추정 매출 상위 25, 하위 25 동을 구해봅시다

In [55]:
high_m_1 = jns.quantile([0.75, 0.25]).iloc[0,0]
high_m_2 = jns.quantile([0.75, 0.25]).iloc[0,1]


low_m_1 = jns.quantile([0.75, 0.25]).iloc[1,0]
low_m_2 = jns.quantile([0.75, 0.25]).iloc[1,1]


In [56]:
high_m_list = jns.query(f'추정매출_1분기>= {high_m_1} and 추정매출_2분기 >= {high_m_2}').읍면동명.unique()
low_m_list = jns.query(f'추정매출_1분기<= {low_m_1} and 추정매출_2분기 <= {low_m_2}').읍면동명.unique()


### 유동인구가 상위 25, 하위 25인 구에 속하는 동

In [62]:
jns_gu_high = jns[jns.시군구명.isin(high_list)]
jns_gu_high

,시군구명,읍면동명,추정매출_1분기,추정매출_2분기,2018년1분기_상권변화지표,2018년2분기_상권변화지표,2019년1분기_상권변화지표,2019년2분기_상권변화지표,2020년1분기_상권변화지표,2020년2분기_상권변화지표
0,강남구,개포2동,12624371008,13182044004,정체,정체,정체,정체,정체,정체
1,강남구,개포4동,10875283987,12128061682,상권축소,상권축소,상권축소,상권축소,상권축소,상권축소
2,강남구,논현1동,48629212194,48167311704,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
3,강남구,논현2동,60419179286,61512365973,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
4,강남구,대치1동,11108924755,11195808375,상권확장,상권확장,다이나믹,다이나믹,다이나믹,다이나믹
...,...,...,...,...,...,...,...,...,...,...
587,서초구,서초3동,110605486018,114263534228,상권축소,상권축소,상권축소,상권축소,정체,정체
588,서초구,서초4동,42131305246,37290195564,다이나믹,다이나믹,다이나믹,다이나믹,상권축소,상권축소
589,서초구,양재1동,44873935416,47260462746,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
590,서초구,양재2동,47713545872,48604420610,상권축소,상권축소,상권축소,상권축소,상권축소,상권축소


In [63]:
jns_gu_low = jns[jns.시군구명.isin(low_list)]

### 유동인구 상위 25인 구, 매출 상위 25인 동

In [65]:
jns_dong_high = jns_gu_high[jns_gu_high.읍면동명.isin(high_m_list)]

In [66]:
jns_dong_low = jns_gu_low[jns_gu_low.읍면동명.isin(low_m_list)]

In [68]:
jns_dong_high.to_csv('귀미한/핫플비핫플_유동인구높매출높.csv', encoding='cp949')

In [69]:
jns_dong_low.to_csv('귀미한/핫플비핫플_유동인구낮매출낮.csv', encoding='cp949')

In [107]:

Image(url= "https://mblogthumb-phinf.pstatic.net/MjAxOTA3MjVfNzQg/MDAxNTY0MDEwODY2MTg5.kCpDyy0gD9R5Ih6Bej_8Z3GdJwgaiTnAmD3zCKo5qFYg.cVdSxqX2lBrI7zBkZGHy3QPD3fk9dactnvujvdHwiOMg.PNG.ckr0118/20190225_134216-3.png?type=w800")

In [91]:
jns_dong_low

,시군구명,읍면동명,추정매출_1분기,추정매출_2분기,2018년1분기_상권변화지표,2018년2분기_상권변화지표,2019년1분기_상권변화지표,2019년2분기_상권변화지표,2020년1분기_상권변화지표,2020년2분기_상권변화지표
36,강북구,번2동,473649431,455190622,상권확장,상권확장,다이나믹,다이나믹,다이나믹,다이나믹
37,강북구,번3동,2270448381,2594880330,상권축소,상권축소,다이나믹,다이나믹,다이나믹,다이나믹
39,강북구,삼양동,1060068748,1163148318,상권축소,상권축소,상권축소,상권축소,상권축소,상권축소
43,강북구,수유2동,1207485388,1465558453,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
45,강북구,우이동,1847204652,1917157252,상권축소,상권축소,다이나믹,다이나믹,다이나믹,다이나믹
...,...,...,...,...,...,...,...,...,...,...
518,도봉구,창5동,1749538005,1817237974,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
735,중랑구,망우3동,866902475,742858822,상권축소,상권축소,다이나믹,다이나믹,상권축소,다이나믹
739,중랑구,면목4동,904589356,1174395300,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
743,중랑구,상봉1동,139860794,155099583,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹


매출이랑 유동인구가 낮은 동은 대부분 상권축소, 정체일거라고 생각했는데 오히려 다이나믹이 더 많아보이네요?

In [94]:
# 상권축소, 정체로만 지표 기록된 지역을 뽑아보면 한군데 뿐!
jns_dong_low[(jns_dong_low['2018년1분기_상권변화지표'].isin(['상권축소', '정체'])) & 
             (jns_dong_low['2018년2분기_상권변화지표'].isin(['상권축소', '정체'])) &
             (jns_dong_low['2019년1분기_상권변화지표'].isin(['상권축소', '정체'])) & 
             (jns_dong_low['2019년2분기_상권변화지표'].isin(['상권축소', '정체'])) &
             (jns_dong_low['2020년1분기_상권변화지표'].isin(['상권축소', '정체'])) & 
             (jns_dong_low['2020년2분기_상권변화지표'].isin(['상권축소', '정체']))          
             
            ].drop_duplicates()

,시군구명,읍면동명,추정매출_1분기,추정매출_2분기,2018년1분기_상권변화지표,2018년2분기_상권변화지표,2019년1분기_상권변화지표,2019년2분기_상권변화지표,2020년1분기_상권변화지표,2020년2분기_상권변화지표
39,강북구,삼양동,1060068748,1163148318,상권축소,상권축소,상권축소,상권축소,상권축소,상권축소


In [96]:
# 올해 지표 기준으로 살펴봐도 한군데 뿐!
jns_dong_low[(jns_dong_low['2020년1분기_상권변화지표'].isin(['상권축소', '정체'])) & 
             (jns_dong_low['2020년2분기_상권변화지표'].isin(['상권축소', '정체']))          
             
            ].drop_duplicates()

,시군구명,읍면동명,추정매출_1분기,추정매출_2분기,2018년1분기_상권변화지표,2018년2분기_상권변화지표,2019년1분기_상권변화지표,2019년2분기_상권변화지표,2020년1분기_상권변화지표,2020년2분기_상권변화지표
39,강북구,삼양동,1060068748,1163148318,상권축소,상권축소,상권축소,상권축소,상권축소,상권축소


In [101]:
# 상권축소, 정체로 한번이라도 지표 기록된 지역을 뽑아보면 다섯군데 뿐!
jns_dong_low[(jns_dong_low['2018년1분기_상권변화지표'].isin(['상권축소', '정체'])) | 
             (jns_dong_low['2018년2분기_상권변화지표'].isin(['상권축소', '정체'])) |
             (jns_dong_low['2019년1분기_상권변화지표'].isin(['상권축소', '정체'])) | 
             (jns_dong_low['2019년2분기_상권변화지표'].isin(['상권축소', '정체'])) |
             (jns_dong_low['2020년1분기_상권변화지표'].isin(['상권축소', '정체'])) | 
             (jns_dong_low['2020년2분기_상권변화지표'].isin(['상권축소', '정체']))          
             
            ].drop_duplicates()

,시군구명,읍면동명,추정매출_1분기,추정매출_2분기,2018년1분기_상권변화지표,2018년2분기_상권변화지표,2019년1분기_상권변화지표,2019년2분기_상권변화지표,2020년1분기_상권변화지표,2020년2분기_상권변화지표
37,강북구,번3동,2270448381,2594880330,상권축소,상권축소,다이나믹,다이나믹,다이나믹,다이나믹
39,강북구,삼양동,1060068748,1163148318,상권축소,상권축소,상권축소,상권축소,상권축소,상권축소
45,강북구,우이동,1847204652,1917157252,상권축소,상권축소,다이나믹,다이나믹,다이나믹,다이나믹
142,도봉구,창3동,1864126165,2171373760,상권축소,상권축소,다이나믹,다이나믹,다이나믹,다이나믹
361,중랑구,망우3동,866902475,742858822,상권축소,상권축소,다이나믹,다이나믹,상권축소,다이나믹
372,중랑구,중화1동,1203728896,1257438224,상권축소,상권축소,다이나믹,다이나믹,상권축소,다이나믹


In [105]:
jns_dong_high[(jns_dong_high['2018년1분기_상권변화지표']== '다이나믹') & 
              (jns_dong_high['2018년2분기_상권변화지표']== '다이나믹') &
              (jns_dong_high['2019년1분기_상권변화지표']== '다이나믹') & 
              (jns_dong_high['2019년2분기_상권변화지표']== '다이나믹') &
              (jns_dong_high['2020년1분기_상권변화지표']== '다이나믹') & 
              (jns_dong_high['2020년2분기_상권변화지표']== '다이나믹')
             ].drop_duplicates()

,시군구명,읍면동명,추정매출_1분기,추정매출_2분기,2018년1분기_상권변화지표,2018년2분기_상권변화지표,2019년1분기_상권변화지표,2019년2분기_상권변화지표,2020년1분기_상권변화지표,2020년2분기_상권변화지표
2,강남구,논현1동,48629212194,48167311704,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
3,강남구,논현2동,60419179286,61512365973,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
5,강남구,대치2동,24342683221,23911189172,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
6,강남구,대치4동,51715018862,52689020300,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
10,강남구,삼성2동,26606930853,27487810002,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
12,강남구,신사동,118934240968,132234697732,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
13,강남구,압구정동,102705088426,114967290130,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
14,강남구,역삼1동,183618195402,176071355533,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
15,강남구,역삼2동,21379636071,22446502747,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹
17,강남구,청담동,47535423774,50485417248,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹,다이나믹


### 정리가 필요합니다
원래 앞서 생각했던 방향은, 핫플레이스의 경우 계속 새로운 사업체가 생겨나고 자주 폐업하는 활발한 변화를 보일것으로 예상했었는데(다이나믹) 
실제로는 비핫플에서도 다이나믹 지표가 보임... 

그렇다면 다이나믹 지표는 활용하지 않는게 낫지않을까..?